In [1]:
! rm -rf sample_data
! wget https://files.grouplens.org/datasets/movielens/ml-20m.zip
! unzip -q /content/ml-20m.zip

--2022-04-11 09:17:47--  https://files.grouplens.org/datasets/movielens/ml-20m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 198702078 (189M) [application/zip]
Saving to: ‘ml-20m.zip’

ml-20m.zip          100%[===================>] 189.50M   126MB/s    in 1.5s    

2022-04-11 09:17:49 (126 MB/s) - ‘ml-20m.zip’ saved [198702078/198702078]



In [2]:
# imports
import os
import shutil
import sys
import numpy as np
from scipy import sparse
import pandas as pd
import bottleneck as bn
from copy import deepcopy
import gc

In [3]:
DATA_DIR = '/content/ml-20m'

In [4]:
raw_data = pd.read_csv(os.path.join(DATA_DIR, 'ratings.csv'), header=0)
print(len(raw_data)) # 20000263
raw_data.head()

20000263


,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [5]:
# Binarize the data (only keep ratings >= 4)
raw_data = raw_data[raw_data['rating'] > 3.5]
print(len(raw_data))
raw_data.head()

9995410


,userId,movieId,rating,timestamp
6,1,151,4.0,1094785734
7,1,223,4.0,1112485573
8,1,253,4.0,1112484940
9,1,260,4.0,1112484826
10,1,293,4.0,1112484703


In [6]:
def get_count(tp, id):
    # Don't do this : tp[id] <- This will return a series with no column index and when you run groupby you'll get a KeyError.
    # Do this : tp[[id]] <- This will return a DataFrame with one column for example if id was 'movieId' then that is the column 
    # that tp[['movieId]] will have.
    
    # playcount_groupbyid will be a DataFrameGroupBy object. In order to see any kind of results, some function must be applied on this object.
    # for e.g. the size() function below.

    playcount_groupbyid = tp[[id]].groupby(id, as_index=False)
    
    # If as_index is not set to False count will be a series oobject. Otherwise it will be a DataFrame object with two
    # columns. If id was 'movieId' then the two columns will be 'movieId' and 'size'.
    count = playcount_groupbyid.size()
    
    return count

In [7]:
def filter_triplets(tp, min_uc=5, min_sc=0):
    # Only keep the triplets for items which were clicked on by at least min_sc users. 
    if min_sc > 0:
        itemcount = get_count(tp, 'movieId')
        # tp = tp[tp['movieId'].isin(itemcount.index[itemcount >= min_sc])]
        tp = tp[tp['movieId'].isin((itemcount['movieId'][itemcount['size'] >= min_sc]).tolist())]
    
    # Only keep the triplets for users who clicked on at least min_uc items
    # After doing this, some of the items will have less than min_uc users, but should only be a small proportion
    if min_uc > 0:
        # Returns a DataFrame containing each userid along with its respective count.
        usercount = get_count(tp, 'userId')
        
        # Do not uncomment. Python will throw an error if the commented line immediately below the current line is run.
        # tp = tp[tp['userId'].isin(usercount.index[usercount >= min_uc])]

        # Use the usercount DataFrame to filter out rows in the original DataFrame. 
        tp = tp[tp['userId'].isin((usercount['userId'][usercount['size'] >= min_uc]).tolist())]
    
    # Update both usercount and itemcount after filtering
    usercount, itemcount = get_count(tp, 'userId'), get_count(tp, 'movieId') 
    return tp, usercount, itemcount

In [8]:
raw_data, user_activity, item_popularity = filter_triplets(raw_data)

In [9]:
sparsity = 1. * raw_data.shape[0] / (user_activity.shape[0] * item_popularity.shape[0])
print("After filtering, there are %d watching events from %d users and %d movies (sparsity: %.3f%%)" % 
      (raw_data.shape[0], user_activity.shape[0], item_popularity.shape[0], sparsity * 100))

After filtering, there are 9990682 watching events from 136677 users and 20720 movies (sparsity: 0.353%)


In [10]:
unique_uid = user_activity.index

np.random.seed(98765)
idx_perm = np.random.permutation(unique_uid.size)
unique_uid = unique_uid[idx_perm]

In [11]:
### create train/validation/test users
n_users = unique_uid.size
n_heldout_users = 10000

tr_users = unique_uid[:(n_users - n_heldout_users * 2)]
vd_users = unique_uid[(n_users - n_heldout_users * 2): (n_users - n_heldout_users)]
te_users = unique_uid[(n_users - n_heldout_users):]

In [12]:
print(len(tr_users),len(vd_users),len(te_users))

116677 10000 10000


In [13]:
train_plays = raw_data.loc[raw_data['userId'].isin(tr_users)]
train_plays.head()

,userId,movieId,rating,timestamp
6,1,151,4.0,1094785734
7,1,223,4.0,1112485573
8,1,253,4.0,1112484940
9,1,260,4.0,1112484826
10,1,293,4.0,1112484703


In [14]:
len(set(train_plays['userId']))

115129

In [15]:
unique_sid = pd.unique(train_plays['movieId'])
unique_sid

array([   151,    223,    253, ..., 126106, 126815, 104085])

In [16]:
show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
profile2id = dict((pid, i) for (i, pid) in enumerate(unique_uid))

In [17]:
pro_dir = os.path.join(DATA_DIR, 'pro_sg')

if not os.path.exists(pro_dir):
    os.makedirs(pro_dir)

with open(os.path.join(pro_dir, 'unique_sid.txt'), 'w') as f:
    for sid in unique_sid:
        f.write('%s\n' % sid)

In [18]:
def split_train_test_proportion(data, test_prop=0.2):
    data_grouped_by_user = data.groupby('userId')
    tr_list, te_list = list(), list()

    np.random.seed(98765)

    for i, (_, group) in enumerate(data_grouped_by_user):
        n_items_u = len(group)

        if n_items_u >= 5:
            idx = np.zeros(n_items_u, dtype='bool')
            idx[np.random.choice(n_items_u, size=int(test_prop * n_items_u), replace=False).astype('int64')] = True

            tr_list.append(group[np.logical_not(idx)])
            te_list.append(group[idx])
        else:
            tr_list.append(group)

        if i % 1000 == 0:
            print("%d users sampled" % i)
            sys.stdout.flush()

    data_tr = pd.concat(tr_list)
    data_te = pd.concat(te_list)
    
    return data_tr, data_te

In [19]:
vad_plays = raw_data.loc[raw_data['userId'].isin(vd_users)]
vad_plays = vad_plays.loc[vad_plays['movieId'].isin(unique_sid)]

In [20]:
vad_plays_tr, vad_plays_te = split_train_test_proportion(vad_plays)

0 users sampled
1000 users sampled
2000 users sampled
3000 users sampled
4000 users sampled
5000 users sampled
6000 users sampled
7000 users sampled
8000 users sampled
9000 users sampled


In [ ]:
def print_groupby(vad_plays):
  data_grouped_by_user = vad_plays.groupby('userId')
  for idx,(_,group) in enumerate(data_grouped_by_user):
    print('index')
    print(idx)
    print('-'*100)
    print('group')
    print(group)
    break
print_groupby(vad_plays) 

In [22]:
test_plays = raw_data.loc[raw_data['userId'].isin(te_users)]
test_plays = test_plays.loc[test_plays['movieId'].isin(unique_sid)]

In [23]:
test_plays_tr, test_plays_te = split_train_test_proportion(test_plays)

0 users sampled
1000 users sampled
2000 users sampled
3000 users sampled
4000 users sampled
5000 users sampled
6000 users sampled
7000 users sampled
8000 users sampled
9000 users sampled


In [24]:
def numerize(tp):
    uid = map(lambda x: profile2id[x], tp['userId'])
    sid = map(lambda x: show2id[x], tp['movieId'])
    return pd.DataFrame(data={'uid': list(uid), 'sid': list(sid)}, columns=['uid', 'sid'])

In [25]:
train_data = numerize(train_plays)
train_data.to_csv(os.path.join(pro_dir, 'train.csv'), index=False)

In [26]:
vad_data_tr = numerize(vad_plays_tr)
vad_data_tr.to_csv(os.path.join(pro_dir, 'validation_tr.csv'), index=False)

In [27]:
vad_data_te = numerize(vad_plays_te)
vad_data_te.to_csv(os.path.join(pro_dir, 'validation_te.csv'), index=False)

In [28]:
test_data_tr = numerize(test_plays_tr)
test_data_tr.to_csv(os.path.join(pro_dir, 'test_tr.csv'), index=False)

In [29]:
test_data_te = numerize(test_plays_te)
test_data_te.to_csv(os.path.join(pro_dir, 'test_te.csv'), index=False)

## Step 2: Load pre-processed data, define the Evaluation Functions
As in this [code](https://github.com/dawenl/vae_cf)

Load the pre-processed data

In [30]:
unique_sid = list()
with open(os.path.join(pro_dir, 'unique_sid.txt'), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())

n_items = len(unique_sid)
print(n_items)

20231


In [31]:
def load_train_data(csv_file):
    tp = pd.read_csv(csv_file)
    n_users = tp['uid'].max() + 1

    rows, cols = tp['uid'], tp['sid']
    data = sparse.csr_matrix((np.ones_like(rows),
                             (rows, cols)), dtype='float64',
                             shape=(n_users, n_items))
    return data

In [32]:
def load_tr_te_data(csv_file_tr, csv_file_te):
    tp_tr = pd.read_csv(csv_file_tr)
    tp_te = pd.read_csv(csv_file_te)

    start_idx = min(tp_tr['uid'].min(), tp_te['uid'].min())
    end_idx = max(tp_tr['uid'].max(), tp_te['uid'].max())

    rows_tr, cols_tr = tp_tr['uid'] - start_idx, tp_tr['sid']
    rows_te, cols_te = tp_te['uid'] - start_idx, tp_te['sid']

    data_tr = sparse.csr_matrix((np.ones_like(rows_tr),
                             (rows_tr, cols_tr)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    data_te = sparse.csr_matrix((np.ones_like(rows_te),
                             (rows_te, cols_te)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    return data_tr, data_te

In [33]:
### load training data
X = load_train_data(os.path.join(pro_dir, 'train.csv'))
XtX=np.array( ( X.transpose() * X).todense()) 
XtXdiag=deepcopy(np.diag(XtX))

In [34]:
### load test data
test_data_tr, test_data_te = load_tr_te_data(
    os.path.join(pro_dir, 'test_tr.csv'),
    os.path.join(pro_dir, 'test_te.csv'))

N_test = test_data_tr.shape[0]
idxlist_test = range(N_test)

Evaluate functions: Normalized discounted cumulative gain (NDCG@k) and Recall@k

In [35]:
def NDCG_binary_at_k_batch(X_pred, heldout_batch, k=100):
    '''
    normalized discounted cumulative gain@k for binary relevance
    ASSUMPTIONS: all the 0's in heldout_data indicate 0 relevance
    '''
    batch_users = X_pred.shape[0]
    idx_topk_part = bn.argpartition(-X_pred, k, axis=1)
    topk_part = X_pred[np.arange(batch_users)[:, np.newaxis],
                       idx_topk_part[:, :k]]
    idx_part = np.argsort(-topk_part, axis=1)
    # X_pred[np.arange(batch_users)[:, np.newaxis], idx_topk] is the sorted
    # topk predicted score
    idx_topk = idx_topk_part[np.arange(batch_users)[:, np.newaxis], idx_part]
    # build the discount template
    tp = 1. / np.log2(np.arange(2, k + 2))

    DCG = (heldout_batch[np.arange(batch_users)[:, np.newaxis],
                         idx_topk].toarray() * tp).sum(axis=1)
    IDCG = np.array([(tp[:min(n, k)]).sum()
                     for n in heldout_batch.getnnz(axis=1)])
    return DCG / IDCG

In [36]:
def Recall_at_k_batch(X_pred, heldout_batch, k=100):
    batch_users = X_pred.shape[0]

    idx = bn.argpartition(-X_pred, k, axis=1)
    X_pred_binary = np.zeros_like(X_pred, dtype=bool)
    X_pred_binary[np.arange(batch_users)[:, np.newaxis], idx[:, :k]] = True

    X_true_binary = (heldout_batch > 0).toarray()
    tmp = (np.logical_and(X_true_binary, X_pred_binary).sum(axis=1)).astype(
        np.float32)
    recall = tmp / np.minimum(k, X_true_binary.sum(axis=1))
    return recall

## Step 3: Training and Evaluation of Higher-order Model 

In [37]:
### functions to create the feature-pairs
def create_list_feature_pairs(XtX, threshold):
    AA= np.triu(np.abs(XtX))
    AA[ np.diag_indices(AA.shape[0]) ]=0.0
    ii_pairs = np.where((AA>threshold)==True)
    return ii_pairs

def create_matrix_Z(ii_pairs, X):
    MM = np.zeros( (len(ii_pairs[0]), X.shape[1]),    dtype=np.float)
    MM[np.arange(MM.shape[0]) , ii_pairs[0]   ]=1.0
    MM[np.arange(MM.shape[0]) , ii_pairs[1]   ]=1.0
    CCmask = 1.0-MM    # see Eq. 8 in the paper
    MM=sparse.csc_matrix(MM.T)
    Z=  X * MM
    Z= (Z == 2.0 )
    Z=Z*1.0
    return [ Z, CCmask]

In [38]:
### training-function of higher-order model
def train_higher(XtX, XtXdiag,lambdaBB, ZtZ, ZtZdiag, lambdaCC, CCmask, ZtX, rho, epochs):
    # precompute for BB
    ii_diag=np.diag_indices(XtX.shape[0])
    XtX[ii_diag] = XtXdiag+lambdaBB
    PP=np.linalg.inv(XtX)
    # precompute for CC
    ii_diag_ZZ=np.diag_indices(ZtZ.shape[0])
    ZtZ[ii_diag_ZZ] = ZtZdiag+lambdaCC+rho
    QQ=np.linalg.inv(ZtZ)
    # initialize
    CC = np.zeros( (ZtZ.shape[0], XtX.shape[0]),dtype=np.float )
    DD = np.zeros( (ZtZ.shape[0], XtX.shape[0]),dtype=np.float )
    UU = np.zeros( (ZtZ.shape[0], XtX.shape[0]),dtype=np.float ) # is Gamma in paper
    for iter in range(epochs):
        print("epoch {}".format(iter))
        # learn BB
        XtX[ii_diag] = XtXdiag
        BB= PP.dot(XtX-ZtX.T.dot(CC))
        gamma = np.diag(BB) / np.diag(PP)
        BB-= PP *gamma
        # learn CC
        CC= QQ.dot(ZtX-ZtX.dot(BB) +rho *(DD-UU))
        # learn DD
        DD=  CC  * CCmask 
        #DD= np.maximum(0.0, DD) # if you want to enforce non-negative parameters
        # learn UU (is Gamma in paper)
        UU+= CC-DD
    return [BB,DD]

train the higher-order model

In [39]:
### choose the training-hyperparameters
epochs = 40

#threshold, lambdaBB, lambdaCC, rho =   1750,  500, 10000, 100000  # ML-20M: 40k of higher orders
threshold, lambdaBB, lambdaCC, rho =   3500,  500,  5000, 100000  # ML-20M: 10k of higher orders
#threshold, lambdaBB, lambdaCC, rho =   6500,  500,  5000, 100000  # ML-20M:  2k of higher orders
#threshold, lambdaBB, lambdaCC, rho =  10000,  500,  2000,  30000  # ML-20M: 500 of higher orders

#threshold, lambdaBB, lambdaCC, rho =  13000, 1000, 30000, 100000  # Nflx: 40k of higher orders
#threshold, lambdaBB, lambdaCC, rho =  22000, 1000, 30000, 100000  # Nflx: 10k of higher orders
#threshold, lambdaBB, lambdaCC, rho =  33000, 1000, 10000, 100000  # Nflx:  2k of higher orders
#threshold, lambdaBB, lambdaCC, rho =  44000, 1000,  3000,  30000  # Nflx: 500 of higher orders

#threshold, lambdaBB, lambdaCC, rho =    750,  200,  1200,  10000  # MSD: 40k of higher orders
#threshold, lambdaBB, lambdaCC, rho =   1850,  200,  1000,  10000  # MSD: 10k of higher orders
#threshold, lambdaBB, lambdaCC, rho =   4050,  200,   200,  10000  # MSD:  2k of higher orders
#threshold, lambdaBB, lambdaCC, rho =   6820,  200,  1200,  10000  # MSD: 500 of higher orders

In [ ]:
### create the list of feature-pairs and the higher-order matrix Z
XtX[ np.diag_indices(XtX.shape[0]) ]=XtXdiag #if code is re-run, ensure that the diagonal is correct
ii_feature_pairs = create_list_feature_pairs(XtX, threshold)
print("number of feature-pairs: {}".format(len(ii_feature_pairs[0])))
Z, CCmask = create_matrix_Z(ii_feature_pairs, X)
Z_test_data_tr , _ = create_matrix_Z(ii_feature_pairs, test_data_tr) 

In [ ]:
### create the higher-order matrices
ZtZ=np.array(  (Z.transpose() * Z).todense()) 
ZtX=np.array( (Z.transpose() * X).todense()) 
ZtZdiag=deepcopy(np.diag(ZtZ))

In [ ]:
### iterative training, and evaluation every 10 epochs 
BB, CC = train_higher(XtX, XtXdiag, lambdaBB, ZtZ, ZtZdiag, lambdaCC, CCmask, ZtX, rho, epochs)

epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39


evaluate the higher-order model

In [ ]:
### evaluation-function of higher-order model
def evaluate_higher(BB,CC,test_data_tr, Z_test_data_tr, N_test, batch_size_test=5000):
    print("Evaluating on test set ...")
    #evaluate in batches
    n100_list, r20_list, r50_list, r10_list = [], [], [], []
    for bnum, st_idx in enumerate(range(0, N_test, batch_size_test)):
        end_idx = min(st_idx + batch_size_test, N_test)
        Xtest = test_data_tr[idxlist_test[st_idx:end_idx]]
        Ztest = Z_test_data_tr[idxlist_test[st_idx:end_idx]]
        if sparse.isspmatrix(Xtest):
            Xtest = Xtest.toarray()
            Ztest = Ztest.toarray()
        Xtest = Xtest.astype('float32')
        Ztest = Ztest.astype('float32')
        pred_val = (Xtest).dot(BB) + Ztest.dot(CC)
        pred_val[Xtest.nonzero()] = -np.inf # exclude examples from training and validation (if any)
        n100_list.append(NDCG_binary_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=100))
        r20_list.append(Recall_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=20))
        r50_list.append(Recall_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=50))
        r10_list.append(Recall_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=10))
    n100_list = np.concatenate(n100_list)
    r20_list = np.concatenate(r20_list)
    r50_list = np.concatenate(r50_list)
    r10_list = np.concatenate(r10_list)
    print("Test Recall@10=%.5f (%.5f)" % (np.mean(r10_list), np.std(r10_list) / np.sqrt(len(r10_list))))
    print("Test Recall@20=%.5f (%.5f)" % (np.mean(r20_list), np.std(r20_list) / np.sqrt(len(r20_list))))
    print("Test Recall@50=%.5f (%.5f)" % (np.mean(r50_list), np.std(r50_list) / np.sqrt(len(r50_list))))
    print("Test NDCG@100=%.5f (%.5f)" % (np.mean(n100_list), np.std(n100_list) / np.sqrt(len(n100_list))))

In [ ]:
evaluate_higher(BB,CC,test_data_tr, Z_test_data_tr, N_test)

Evaluating on test set ...
Test Recall@10=0.34310 (0.00264)
Test Recall@20=0.39999 (0.00270)
Test Recall@50=0.53025 (0.00282)
Test NDCG@100=0.42923 (0.00215)
